# Boulder Precinct Choropleth Map ETL
    
Create data files required for a choropleth map showing values by Boulder precincts.
- Transform the input data to file formats compatible with Folium choropleth and Leaflet maps
- Save precinct geometry files that  Folium Choropleth() can read as geometry
    - GeoJson Feature Collection with Feature id=precinct_name
- Started: August 10, 2022
- Updated: December 16, 2024

##  PREFERRED METHOD TO CREATE FOLIUM CHOROPLETH LAYER

### Create choropleth layer with gdf for geometry and df for data
```
folium.Choropleth(
    geo_data = <geodataframe>,
    key_on = 'feature.properties.<geodata_key>', # defines key column in 'geo_data'
    data = <dataframe>,
    columns = ['<dataframe_key>', '<dataframe_column>'], # columns[0], dataframe_key, defines key column in 'data'
    fill_color = "YlGn",
    fill_opacity = 0.7,
    line_opacity = 0.2,
    name = '<layer_name>',
    legend_name = '<legend_name>'
)

```

## Input Files

- Boulder County Precincts
    - Source of Truth: Shape files downloaded from Boulder County on December 15, 2024
    - https://opendata-bouldercounty.hub.arcgis.com/datasets/c8e2897d283b47f780920af0827d5126_0/explore?location=40.086460%2C-105.373100%2C10.51

## Output Files

Leaflet map file with choropleth layer:

## Issues
- Folium.Choropleth does not show tooltips or popups.
- Folium.Choropleth() is very picky about the data in the geometry file
    - needs feature_id=precinct_name to merge choropleth data layers


In [1]:
import pandas as pd
import geopandas as gpd
import folium

### Precincts - Boulder County Precinct Boundaries

In [ ]:
pctgdf = gpd.read_file('data/pct/Precincts.shp', columns=['PRECINCT','geometry'])
pctgdf = pctgdf.rename(columns={'PRECINCT':'precinct'})
pctgdf['precinct'] = pctgdf['precinct'].astype('int64')

# convert crs to WGS 84, ESPG:4326
pctgdf.to_crs(4326, inplace=True)
pctgdf.info()

In [ ]:
pctgdf.head(4)

## ETL Election results by precinct data

Create dataframe with rows of precincts and results in columns
For Folium Choropleth()
- Create a column named "precinct_name"

### Election result value columns
- Turnout by precinct
- Votes by precinct
-   

In [ ]:
# get election votes results data
pres = pd.read_csv('data/election_results/election_results_2024_president_boulder.csv')
pres.info()

In [ ]:
# get turnout data
turnout = pd.read_csv('data/election_results/Vote Data Results Active Only(Precinct).csv', dtype=str)
turnout = turnout[['Precinct - Full', 'Target 92+ Voted %']]
turnout.info()
turnout.head(4)

In [ ]:
#########   CHOROPLETH MAP PREFERRED METHOD GeoDataFrame + DataFrame   #####################
# Create Choropleth layer showing Dem votes for President
# use GeoDataFrame for geometry
# use pres dataframe for data
# NOTE: preliminary voting results

# Create basemap layer that shows Boulder County
m = folium.Map(tiles='cartodb positron', location=(40.1, -105.3), zoom_level=10)

folium.Choropleth(
    geo_data=pctgdf,
    key_on="feature.properties.precinct",
    data=pres,
    highlight=True,
    columns=["precinct_name", "dem_votes_pres"],
    tooltip = 'tooltip',
    popup = 'popup',
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Dem Votes for President",
    name="Presidential Dem Votes",
).add_to(m)

folium.LayerControl().add_to(m)

m.save('dem_votes_pres.html')

# Display the Choropleth
m

In [ ]:
# Create Choropleth map showing Dem Margin for President
# NOTE: preliminary voting results

# Create basemap layer that shows Boulder County
m = folium.Map(tiles='cartodb positron', location=(40.1, -105.3), zoom_level=10)

folium.Choropleth(
    geo_data=pctgdf,
    name="Presidential Dem Margin",
    data=pres,
    columns=["precinct_name", "dem_margin_pres"],
    key_on="feature.properties.precinct",
    highlight=True,
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Dem Margin for President",
).add_to(m)

folium.LayerControl().add_to(m)

m.save('dem_margin.html')

# Display the Choropleth
m

########  Try with final data from Geoff  ###########

Needs a lot of data cleaning